In [1]:
# Importing necessary packages
import pandas as pd
import numpy as np
import statsmodels.api as sm

**If using Stargazer to export LaTeX code:** \
!pip install Stargazer \
from stargazer.stargazer import Stargazer

In [2]:
data = pd.read_csv('factordata.csv')

# Multiplying by Indicators
data['IndGMB_U'] = data['GMB_U'] * data['Method_Estimated']
data['IndGMB_S'] = data['GMB_S'] * data['Method_Estimated']
data['IndMkt-RF'] = data['Mkt-RF'] * data['Method_Estimated']
data['IndSMB'] = data['SMB'] * data['Method_Estimated']
data['IndHML'] = data['HML'] * data['Method_Estimated']
data['IndRMW'] = data['RMW'] * data['Method_Estimated']
data['IndCMA'] = data['CMA'] * data['Method_Estimated']

print(data)

           cusip  year  Month  monthyear  Mkt-RF   SMB   HML    RF   RMW  \
0       46603210  2015      1     201501   -3.11 -0.92 -3.59  0.00  1.61   
1       46603210  2015      2     201502    6.13  0.32 -1.86  0.00 -1.12   
2       46603210  2015      3     201503   -1.12  3.07 -0.38  0.00  0.09   
3       46603210  2015      4     201504    0.59 -3.09  1.82  0.00  0.06   
4       46603210  2015      5     201505    1.36  0.84 -1.15  0.00 -1.80   
...          ...   ...    ...        ...     ...   ...   ...   ...   ...   
295824  12503M10  2023      8     202308   -2.39 -3.68 -1.08  0.45  3.42   
295825  12503M10  2023      9     202309   -5.24 -1.79  1.45  0.43  1.85   
295826  12503M10  2023     10     202310   -3.18 -4.05  0.19  0.47  2.47   
295827  12503M10  2023     11     202311    8.83 -0.11  1.66  0.44 -3.81   
295828  12503M10  2023     12     202312    4.87  7.33  4.92  0.43 -3.04   

         CMA  ...  Method_Estimated     GMB_U     GMB_S  IndGMB_U  IndGMB_S  \
0      -

In [3]:
# Get sector dummies!
sectordummies = pd.get_dummies(data['sector'], dtype = int, prefix='sector')
# Concatenate dummy variables with main DataFrame
withsectors = pd.concat([data, sectordummies], axis=1)

# Multiplying by Indicators (REMOVING RETAIL (median))
withsectors['IndU_Acc'] = withsectors['GMB_U'] * withsectors['sector_AccomFood']
withsectors['IndU_Cons'] = withsectors['GMB_U'] * withsectors['sector_Construction']
withsectors['IndU_Fin'] = withsectors['GMB_U'] * withsectors['sector_FinanceIns']
withsectors['IndU_HC'] = withsectors['GMB_U'] * withsectors['sector_Healthcare']
withsectors['IndU_Info'] = withsectors['GMB_U'] * withsectors['sector_Information']
withsectors['IndU_Man'] = withsectors['GMB_U'] * withsectors['sector_Manufacturing']
withsectors['IndU_Oil'] = withsectors['GMB_U'] * withsectors['sector_OilGas']
withsectors['IndU_RE'] = withsectors['GMB_U'] * withsectors['sector_RealEstate']
withsectors['IndU_Serv'] = withsectors['GMB_U'] * withsectors['sector_Services']
withsectors['IndU_Transp'] = withsectors['GMB_U'] * withsectors['sector_TransportWarehouse']
withsectors['IndU_Util'] = withsectors['GMB_U'] * withsectors['sector_Utilities']
withsectors['IndU_Whol'] = withsectors['GMB_U'] * withsectors['sector_Wholesale']

withsectors['IndS_Acc'] = withsectors['GMB_S'] * withsectors['sector_AccomFood']
withsectors['IndS_Cons'] = withsectors['GMB_S'] * withsectors['sector_Construction']
withsectors['IndS_Fin'] = withsectors['GMB_S'] * withsectors['sector_FinanceIns']
withsectors['IndS_HC'] = withsectors['GMB_S'] * withsectors['sector_Healthcare']
withsectors['IndS_Info'] = withsectors['GMB_S'] * withsectors['sector_Information']
withsectors['IndS_Man'] = withsectors['GMB_S'] * withsectors['sector_Manufacturing']
withsectors['IndS_Oil'] = withsectors['GMB_S'] * withsectors['sector_OilGas']
withsectors['IndS_RE'] = withsectors['GMB_S'] * withsectors['sector_RealEstate']
withsectors['IndS_Serv'] = withsectors['GMB_S'] * withsectors['sector_Services']
withsectors['IndS_Transp'] = withsectors['GMB_S'] * withsectors['sector_TransportWarehouse']
withsectors['IndS_Util'] = withsectors['GMB_S'] * withsectors['sector_Utilities']
withsectors['IndS_Whol'] = withsectors['GMB_S'] * withsectors['sector_Wholesale']

In [4]:
# Get time dummies!
yeardummies = pd.get_dummies(data['year'], dtype = int, prefix='year')
# Drop first year (2002) to avoid multicollinearity with time dummies
yeardummies.drop('year_2002', axis=1, inplace=True)
# Concatenate dummy variables with main DataFrame
factordata = pd.concat([withsectors, yeardummies], axis=1)

# Removing Salient Sectors

In [5]:
factordata_minus_salient = factordata.copy()

# Drop rows where 'sector' is equal to a salient industry
factordata_minus_salient = factordata_minus_salient[factordata_minus_salient['sector'] != 'OilGas']
factordata_minus_salient = factordata_minus_salient[factordata_minus_salient['sector'] != 'Utilities']
factordata_minus_salient = factordata_minus_salient[factordata_minus_salient['sector'] != 'Transportation']

# Table 1: Baseline

**Just the FF3**

In [6]:
T1X1 = sm.add_constant(factordata_minus_salient[['Mkt-RF', 'SMB', 'HML',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T1y1 = factordata_minus_salient["ExcessReturn"]
T1model1 = sm.OLS(T1y1, T1X1).fit(cov_type='HC1')
print(T1model1.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.512
Model:                            OLS   Adj. R-squared:                  0.512
Method:                 Least Squares   F-statistic:                 1.489e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:56:16   Log-Likelihood:             1.6152e+05
No. Observations:              269527   AIC:                        -3.230e+05
Df Residuals:                  269502   BIC:                        -3.227e+05
Df Model:                          24                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1287      0.002    -58.011      0.0

**Just the FF5**

In [7]:
T1X2 = sm.add_constant(factordata_minus_salient[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T1y2 = factordata_minus_salient["ExcessReturn"]
T1model2 = sm.OLS(T1y2, T1X2).fit(cov_type='HC1')
print(T1model2.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.513
Model:                            OLS   Adj. R-squared:                  0.513
Method:                 Least Squares   F-statistic:                 1.382e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:56:16   Log-Likelihood:             1.6182e+05
No. Observations:              269527   AIC:                        -3.236e+05
Df Residuals:                  269500   BIC:                        -3.233e+05
Df Model:                          26                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1235      0.002    -54.077      0.0

**FF3 + GMB_U**

In [8]:
T1X3 = sm.add_constant(factordata_minus_salient[['Mkt-RF', 'SMB', 'HML', 'GMB_U',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T1y3 = factordata_minus_salient["ExcessReturn"]
T1model3 = sm.OLS(T1y3, T1X3).fit(cov_type='HC1')
print(T1model3.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.514
Model:                            OLS   Adj. R-squared:                  0.514
Method:                 Least Squares   F-statistic:                 1.424e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:56:17   Log-Likelihood:             1.6207e+05
No. Observations:              269527   AIC:                        -3.241e+05
Df Residuals:                  269501   BIC:                        -3.238e+05
Df Model:                          25                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1291      0.002    -58.010      0.0

**FF5 + GMB_U**

In [9]:
T1X4 = sm.add_constant(factordata_minus_salient[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'GMB_U',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T1y4 = factordata_minus_salient["ExcessReturn"]
T1model4 = sm.OLS(T1y4, T1X4).fit(cov_type='HC1')
print(T1model4.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.516
Model:                            OLS   Adj. R-squared:                  0.516
Method:                 Least Squares   F-statistic:                 1.324e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:56:17   Log-Likelihood:             1.6266e+05
No. Observations:              269527   AIC:                        -3.253e+05
Df Residuals:                  269499   BIC:                        -3.250e+05
Df Model:                          27                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1175      0.002    -51.024      0.0

**FF3 + GMB_S**

In [10]:
T1X5 = sm.add_constant(factordata_minus_salient[['Mkt-RF', 'SMB', 'HML', 'GMB_S',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T1y5 = factordata_minus_salient["ExcessReturn"]
T1model5 = sm.OLS(T1y5, T1X5).fit(cov_type='HC1')
print(T1model5.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.514
Model:                            OLS   Adj. R-squared:                  0.514
Method:                 Least Squares   F-statistic:                 1.429e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:56:19   Log-Likelihood:             1.6189e+05
No. Observations:              269527   AIC:                        -3.237e+05
Df Residuals:                  269501   BIC:                        -3.235e+05
Df Model:                          25                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1299      0.002    -58.506      0.0

**FF5 + GMB_S**

In [11]:
T1X6 = sm.add_constant(factordata_minus_salient[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA','GMB_S',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T1y6 = factordata_minus_salient["ExcessReturn"]
T1model6 = sm.OLS(T1y6, T1X6).fit(cov_type='HC1')
print(T1model6.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.516
Model:                            OLS   Adj. R-squared:                  0.516
Method:                 Least Squares   F-statistic:                 1.328e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:56:21   Log-Likelihood:             1.6250e+05
No. Observations:              269527   AIC:                        -3.249e+05
Df Residuals:                  269499   BIC:                        -3.246e+05
Df Model:                          27                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1204      0.002    -52.377      0.0

In [12]:
# T1stargazer = Stargazer([T1model1, T1model2, T1model3, T1model4, T1model5, T1model6])
# print(T1stargazer.render_latex())

# Table 2: How does the GMB Factor Change with the Sector?

## Table 2a: GMB_U

**(1) FF3 + GMB_U + (Sector Indicators)GMB_U + Time Fixed Effects**

In [13]:
T2aX1 = sm.add_constant(factordata_minus_salient[['Mkt-RF', 'SMB', 'HML', 'GMB_U', 
                          'IndU_Acc', 'IndU_Cons', 'IndU_Fin', 'IndU_HC', 'IndU_Info', 'IndU_Man', 
                          'IndU_RE', 'IndU_Serv', 'IndU_Whol',
                          'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                          'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                          'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                          'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T2ay1 = factordata_minus_salient["ExcessReturn"]
T2amodel1 = sm.OLS(T2ay1, T2aX1).fit(cov_type='HC1')
print(T2amodel1.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.517
Model:                            OLS   Adj. R-squared:                  0.517
Method:                 Least Squares   F-statistic:                 1.055e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:56:24   Log-Likelihood:             1.6288e+05
No. Observations:              269527   AIC:                        -3.257e+05
Df Residuals:                  269492   BIC:                        -3.253e+05
Df Model:                          34                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1291      0.002    -57.905      0.0

**(2) FF3 + GMB_U + (Sector Indicators)GMB_U + Time Fixed Effects + SECTOR FIXED EFFECTS**

In [14]:
T2aX2 = sm.add_constant(factordata_minus_salient[['Mkt-RF', 'SMB', 'HML', 'GMB_U', 
                          'IndU_Acc', 'IndU_Cons', 'IndU_Fin', 'IndU_HC', 'IndU_Info', 'IndU_Man', 
                          'IndU_RE', 'IndU_Serv', 'IndU_Whol',
                          'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                          'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                          'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                          'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023',
                          'sector_AccomFood', 'sector_Construction', 'sector_FinanceIns', 'sector_Healthcare', 
                          'sector_Information', 'sector_Manufacturing', 'sector_RealEstate', 
                          'sector_Services', 'sector_Wholesale']])
T2ay2 = factordata_minus_salient["ExcessReturn"]
T2amodel2 = sm.OLS(T2ay2, T2aX2).fit(cov_type='HC1')
print(T2amodel2.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.517
Model:                            OLS   Adj. R-squared:                  0.517
Method:                 Least Squares   F-statistic:                     8368.
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:56:27   Log-Likelihood:             1.6290e+05
No. Observations:              269527   AIC:                        -3.257e+05
Df Residuals:                  269483   BIC:                        -3.253e+05
Df Model:                          43                                         
Covariance Type:                  HC1                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.1287 

**(3) FF5 + GMB_U + (Sector Indicators)GMB_U + Time Fixed Effects**

In [15]:
T2aX3 = sm.add_constant(factordata_minus_salient[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'GMB_U', 
                          'IndU_Acc', 'IndU_Cons', 'IndU_Fin', 'IndU_HC', 'IndU_Info', 'IndU_Man', 
                          'IndU_RE', 'IndU_Serv', 'IndU_Whol',
                          'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                          'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                          'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                          'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T2ay3 = factordata_minus_salient["ExcessReturn"]
T2amodel3 = sm.OLS(T2ay3, T2aX3).fit(cov_type='HC1')
print(T2amodel3.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.519
Model:                            OLS   Adj. R-squared:                  0.519
Method:                 Least Squares   F-statistic:                 1.001e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:56:30   Log-Likelihood:             1.6348e+05
No. Observations:              269527   AIC:                        -3.269e+05
Df Residuals:                  269490   BIC:                        -3.265e+05
Df Model:                          36                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1176      0.002    -50.983      0.0

**(4) FF5 + GMB_U + (Sector Indicators)GMB_U + Time Fixed Effects + SECTOR FIXED EFFECTS**

In [16]:
T2aX4 = sm.add_constant(factordata_minus_salient[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'GMB_U', 
                          'IndU_Acc', 'IndU_Cons', 'IndU_Fin', 'IndU_HC', 'IndU_Info', 'IndU_Man', 
                          'IndU_RE', 'IndU_Serv', 'IndU_Whol',
                          'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                          'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                          'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                          'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023',
                          'sector_AccomFood', 'sector_Construction', 'sector_FinanceIns', 'sector_Healthcare', 
                          'sector_Information', 'sector_Manufacturing', 'sector_RealEstate', 
                          'sector_Services', 'sector_Wholesale']])
T2ay4 = factordata_minus_salient["ExcessReturn"]
T2amodel4 = sm.OLS(T2ay4, T2aX4).fit(cov_type='HC1')
print(T2amodel4.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.519
Model:                            OLS   Adj. R-squared:                  0.519
Method:                 Least Squares   F-statistic:                     8030.
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:56:33   Log-Likelihood:             1.6349e+05
No. Observations:              269527   AIC:                        -3.269e+05
Df Residuals:                  269481   BIC:                        -3.264e+05
Df Model:                          45                                         
Covariance Type:                  HC1                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.1173 

In [17]:
# T2astargazer = Stargazer([T2amodel1, T2amodel2, T2amodel3, T2amodel4])
# print(T2astargazer.render_latex())

## Table 2b: GMB_S

**(1) FF3 + GMB_S + (Sector Indicators)GMB_S + Time Fixed Effects**

In [18]:
T2bX1 = sm.add_constant(factordata_minus_salient[['Mkt-RF', 'SMB', 'HML', 'GMB_S', 
                          'IndS_Acc', 'IndS_Cons', 'IndS_Fin', 'IndS_HC', 'IndS_Info', 'IndS_Man', 
                          'IndS_RE', 'IndS_Serv', 'IndS_Whol',
                          'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                          'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                          'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                          'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T2by1 = factordata_minus_salient["ExcessReturn"]
T2bmodel1 = sm.OLS(T2by1, T2bX1).fit(cov_type='HC1')
print(T2bmodel1.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.515
Model:                            OLS   Adj. R-squared:                  0.515
Method:                 Least Squares   F-statistic:                 1.055e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:56:35   Log-Likelihood:             1.6220e+05
No. Observations:              269527   AIC:                        -3.243e+05
Df Residuals:                  269492   BIC:                        -3.240e+05
Df Model:                          34                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1298      0.002    -58.544      0.0

**(2) FF3 + GMB_S + (Sector Indicators)GMB_S + Time Fixed Effects + SECTOR Fixed Effects**

In [19]:
T2bX2 = sm.add_constant(factordata_minus_salient[['Mkt-RF', 'SMB', 'HML', 'GMB_S', 
                          'IndS_Acc', 'IndS_Cons', 'IndS_Fin', 'IndS_HC', 'IndS_Info', 'IndS_Man', 
                          'IndS_RE', 'IndS_Serv', 'IndS_Whol',
                          'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                          'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                          'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                          'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023',
                          'sector_AccomFood', 'sector_Construction', 'sector_FinanceIns', 'sector_Healthcare', 
                          'sector_Information', 'sector_Manufacturing', 'sector_RealEstate', 
                          'sector_Services', 'sector_Wholesale']])
T2by2 = factordata_minus_salient["ExcessReturn"]
T2bmodel2 = sm.OLS(T2by2, T2bX2).fit(cov_type='HC1')
print(T2bmodel2.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.515
Model:                            OLS   Adj. R-squared:                  0.515
Method:                 Least Squares   F-statistic:                     8364.
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:56:38   Log-Likelihood:             1.6222e+05
No. Observations:              269527   AIC:                        -3.243e+05
Df Residuals:                  269483   BIC:                        -3.239e+05
Df Model:                          43                                         
Covariance Type:                  HC1                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.1309 

**(3) FF5 + GMB_S + (Sector Indicators)GMB_S + Time Fixed Effects**

In [20]:
T2bX3 = sm.add_constant(factordata_minus_salient[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'GMB_S', 
                          'IndS_Acc', 'IndS_Cons', 'IndS_Fin', 'IndS_HC', 'IndS_Info', 'IndS_Man', 
                          'IndS_RE', 'IndS_Serv', 'IndS_Whol',  
                          'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                          'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                          'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                          'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T2by3 = factordata_minus_salient["ExcessReturn"]
T2bmodel3 = sm.OLS(T2by3, T2bX3).fit(cov_type='HC1')
print(T2bmodel3.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.517
Model:                            OLS   Adj. R-squared:                  0.517
Method:                 Least Squares   F-statistic:                     9995.
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:56:39   Log-Likelihood:             1.6279e+05
No. Observations:              269527   AIC:                        -3.255e+05
Df Residuals:                  269490   BIC:                        -3.251e+05
Df Model:                          36                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1204      0.002    -52.409      0.0

**(4) FF5 + GMB_S + (Sector Indicators)GMB_U + Time Fixed Effects + SECTOR FIXED EFFECTS**

In [21]:
T2bX4 = sm.add_constant(factordata_minus_salient[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'GMB_S', 
                          'IndS_Acc', 'IndS_Cons', 'IndS_Fin', 'IndS_HC', 'IndS_Info', 'IndS_Man', 
                          'IndS_RE', 'IndS_Serv', 'IndS_Whol',
                          'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                          'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                          'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                          'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023',
                          'sector_AccomFood', 'sector_Construction', 'sector_FinanceIns', 'sector_Healthcare', 
                          'sector_Information', 'sector_Manufacturing', 'sector_RealEstate', 
                          'sector_Services', 'sector_Wholesale']])
T2by4 = factordata_minus_salient["ExcessReturn"]
T2bmodel4 = sm.OLS(T2by4, T2bX4).fit(cov_type='HC1')
print(T2bmodel4.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.517
Model:                            OLS   Adj. R-squared:                  0.517
Method:                 Least Squares   F-statistic:                     8020.
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:56:39   Log-Likelihood:             1.6281e+05
No. Observations:              269527   AIC:                        -3.255e+05
Df Residuals:                  269481   BIC:                        -3.250e+05
Df Model:                          45                                         
Covariance Type:                  HC1                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.1215 

In [22]:
# T2bstargazer = Stargazer([T2bmodel1, T2bmodel2, T2bmodel3, T2bmodel4])
# print(T2bstargazer.render_latex())

# Table 3: Estimated vs. Reported Emissions

**FF3 + GMB_U + (Method Indicator)GMB_U + Method Ind.**

In [23]:
T3X1 = sm.add_constant(factordata_minus_salient[['Mkt-RF', 'SMB', 'HML', 'Method_Estimated', 'GMB_U', 'IndGMB_U',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T3y1 = factordata_minus_salient["ExcessReturn"]
T3model1 = sm.OLS(T3y1, T3X1).fit(cov_type='HC1')
print(T3model1.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.516
Model:                            OLS   Adj. R-squared:                  0.516
Method:                 Least Squares   F-statistic:                 1.329e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:56:40   Log-Likelihood:             1.6252e+05
No. Observations:              269527   AIC:                        -3.250e+05
Df Residuals:                  269499   BIC:                        -3.247e+05
Df Model:                          27                                         
Covariance Type:                  HC1                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.1284      0.002  

**FF5 + GMB_U + (Method Indicator)GMB_U + Method Ind.**

In [24]:
T3X2 = sm.add_constant(factordata_minus_salient[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Method_Estimated', 'GMB_U', 'IndGMB_U',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T3y2 = factordata_minus_salient["ExcessReturn"]
T3model2 = sm.OLS(T3y2, T3X2).fit(cov_type='HC1')
print(T3model2.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.518
Method:                 Least Squares   F-statistic:                 1.243e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:56:40   Log-Likelihood:             1.6312e+05
No. Observations:              269527   AIC:                        -3.262e+05
Df Residuals:                  269497   BIC:                        -3.259e+05
Df Model:                          29                                         
Covariance Type:                  HC1                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.1167      0.002  

**FF3 + GMB_S + (Method Indicator)GMB_S + Method Ind.**

In [25]:
T3X3 = sm.add_constant(factordata_minus_salient[['Mkt-RF', 'SMB', 'HML', 'Method_Estimated', 'GMB_S', 'IndGMB_S',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T3y3 = factordata_minus_salient["ExcessReturn"]
T3model3 = sm.OLS(T3y3, T3X3).fit(cov_type='HC1')
print(T3model3.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.514
Model:                            OLS   Adj. R-squared:                  0.514
Method:                 Least Squares   F-statistic:                 1.329e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:56:41   Log-Likelihood:             1.6200e+05
No. Observations:              269527   AIC:                        -3.239e+05
Df Residuals:                  269499   BIC:                        -3.237e+05
Df Model:                          27                                         
Covariance Type:                  HC1                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.1306      0.002  

**FF3 + GMB_S + (Method Indicator)GMB_S + Method Ind.**

In [26]:
T3X4 = sm.add_constant(factordata_minus_salient[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Method_Estimated', 'GMB_S', 'IndGMB_S',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
T3y4 = factordata_minus_salient["ExcessReturn"]
T3model4 = sm.OLS(T3y4, T3X4).fit(cov_type='HC1')
print(T3model4.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.516
Model:                            OLS   Adj. R-squared:                  0.516
Method:                 Least Squares   F-statistic:                 1.241e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:56:41   Log-Likelihood:             1.6261e+05
No. Observations:              269527   AIC:                        -3.252e+05
Df Residuals:                  269497   BIC:                        -3.248e+05
Df Model:                          29                                         
Covariance Type:                  HC1                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.1211      0.002  

In [27]:
# T3stargazer = Stargazer([T3model1, T3model2, T3model3, T3model4])
# print(T3stargazer.render_latex())